---

[//]: # "Header"

<p style="text-align: left;  font-size:18pt; LINE-HEIGHT:30px;">
    <span style="float: left">
     Technische Hochschule Ingolstadt<br>
     Prof. Dr. Sören Gröttrup <br>
     Laura Dietl
    </span>
    <span style="float: right;">
       Machine Learning 2<br>
        <span style="float: right;">SS 2025</span>
    </span>
</p>

[//]: # "Header Aufgabenblatt"
<br>
<p style="text-align: center;  font-size:18pt; LINE-HEIGHT:30px;">
     <span style="font-weight: bold;">Aufgabenblatt 2</span><br>
     Themen: Optimizer, Momentum, Adam<br>
     Abgabetermin: 24.04.2025, 23:59 Uhr <br>
     Punkte: 15 (+3)
</p>


---

**Namen:** Emelie Hauck, Conrad Christoph

---

### Daten

Der Datensatz aus https://www.kaggle.com/wkirgsn/electric-motor-temperature enthält mehrere Sensormessungen eines "permanent magnet synchronous motor (PMSM)" (Elektromotors) in einem Auto gemessen auf einem Prüfstand. Der PMSM repräsentiert das Prototypmpdell der deutschen OEM's und gesammelt von der Universität Paderborn. Weitere Informationen finden Sie dazu im obigen Link .

Ein interessantes Target Feature ist u.A. die Rotortemperatur `pm`, da diese in kommerziellen Autos nicht gesichert und ökonomisch gemessen werden können.

Das File `electric_motor_temp_data_norm` enthält bereits den normierten und aufbereiteten Datensatz, geplittet in Training `train_data` und Test `test_data`. Die Daten liegen jeweils bereits als TensorDataset bereit.

## Aufgabe 1  [15 Punkte]


1. Laden Sie die Daten `electric_motor_temp_data_norm` und speichern Sie den Tensor der Trainingsdaten in einer Variablen `train_data` und den Tensor der Testdaten in einer variable `test_data`. Welche Größe hat der Trainingsdatensatz?  _Hinweis_: Die Daten können Sie mit der Funktion `torch.load()` einlesen. Sie erhalten dann ein Dictionary, in welchem sich Training- udn Testdaten befinden.
1. Schreiben Sie einen Routine, werlche das unten angegebene Neuronale Netz `Net()` trainiert. Dabei soll während des Trainings der RMSE des jeweiligen Batches in regelmäßigem Abstand berechnet und abgespeichert werden (alle `log_intervall` Iterationen). Weiter soll nach jeder Epoche der RMSE des gesamten Testdatensatzes berechnet und abgespeichert werden.

1. Trainieren Sie das Neuronale Netz mittels SGD für eine Batch Size 256. Was ist der RMSE nach 10 Epochen für die Trainings- bzw. Testdaten?
1. Informieren Sie sich, wie man in PyTorch einen Momentum Term zum SGD hinzufügen kann und trainieren Sie das Netz `Net()` mittels SGD mit Momentum für die Momentumkoeffizienten 0.5, 0.8, 0.9 und 0.99. Vergleichen Sie das Ergebnis und den Verlauf des Lernens mit dem Training ohne Momentum. Was ist jetzt der RMSE nach 10 Epochen?
1. Informieren Sie sich über die Methoden `RMSprop` und `Adam` in PyTorch. Trainieren Sie dann das Neuronale Netz mittels RMSProp- und Adam-Verfahren (und den Standardwerten). Vergleichen Sie das Ergebnis mit den beiden vorherigen Methoden.

_Hinweis:_ 

* Sollte das Training Ihnen zu lange dauer, erstellen Sie sich ein Sample (ca. 10000 Datenpunkte) aus dem Trainingsdatensatz und benutzen Sie dieses während der Implementierung Ihrer Routinen, um zu verifizieren, ob Ihr Algorithmus fehlerfrei durchläuft. Erst am Ende lassen Sie den Algorithmus auf den gesamten Datensatz los.

_Optional (Speed-Up the Training)_ [3 Zusatzpunkte]:
* Ein DataLoader splittet den Datensatz in Mini-Batches. Während des Trainings werden dabei aber die einzelnen Sample für einen Batch einzeln geladen. Dies kann bei Daten in Tabellenform das Training unnötig verlangsamen. Implementieren Sie also eine Funktion `generate_batches(batch_size, n_samples)`, welche bei Übergabe einer Batchsize und der Größe des Datensatzes eine Liste aus Arrays aus Indices erzeugt und zurückgibt, welche die einzelnen Batches beschreibt. Fügen Sie diese Methode in Ihrem Trainings-Algorithmus mit ein. _Beispiel Output:_ `[array([4, 8, 2]), array([0, 7, 3]), array([5, 6, 1])]` Bei einem Datensatz der Größe 9 sind im Ersten Batch die Sample mit den Indices 4,8,2, im zweiten die Sample mit den Indices 0,7,3, usw.



#### Laden benötigter Pakete

In [2]:
# Packages
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as net
import torch.nn.functional as F
import torch.optim as optim

import time

plt.style.use('ggplot')

#### Define Neuronal Net

In [3]:
# Definition of the net structure
class Net(net.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = net.Linear(8, 32)
        self.fc2 = net.Linear(32, 12)
        self.fc3 = net.Linear(12, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### Initialisierung von Parametern

In [4]:
# Define Parameters
n_epochs = 10

batch_size = 256

learning_rate = 0.01
log_interval = 500

random_seed = 1
torch.manual_seed(random_seed)

## Beginn Ihrer Lösung hier

In [5]:
#Aufgabe 1.1

data = torch.load("./Daten Blatt 2/electric_motor_temp_data_norm")

train_data = data["train_data"]
test_data = data["test_data"]

print("Größe des Trainingsdatensatzes:", len(train_data))

#Aufgabe 1.2

def train_model(network, train_data, test_data, optimizer, criterion, 
                batch_size, epochs, log_interval):

    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True) #Mini-Batches
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

    batch_rmse_log = []
    test_rmse_log = []

    for epoch in range(epochs):
        network.train()
        for batch_idx, (x, y) in enumerate(train_loader):

            optimizer.zero_grad()
            output = network(x)
            loss = criterion(output, y.unsqueeze(1))
            loss.backward()
            optimizer.step()

            if batch_idx % log_interval == 0:
                rmse = torch.sqrt(loss).item()
                batch_rmse_log.append(rmse)
                print(f"[Epoche {epoch+1} | Batch Nr. {batch_idx}] RMSE (Train): {rmse:.4f}") #RMSE der Batches im log_interval

        network.eval()
        test_loss = 0
        with torch.no_grad():
            for x_test, y_test in test_loader:
                output = network(x_test)
                test_loss += criterion(output, y_test.unsqueeze(1)).item()
            
            test_loss /= len(test_loader)
            test_rmse = np.sqrt(test_loss)
            test_rmse_log.append(test_rmse)
            print(f"==> Epoche {epoch+1}: RMSE (Test): {test_rmse:.4f}") #RMSE pro Epoche von Testdatensatz

    return batch_rmse_log, test_rmse_log


#Aufgabe 1.3
network = Net()
criterion = net.MSELoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate)

batch_rmse_log, test_rmse_log = train_model(network, train_data, test_data, optimizer, criterion, batch_size=batch_size, epochs=n_epochs, log_interval=log_interval)

print("RMSE nach 10 Epochen:")
print(f"Trainings-RMSE: {batch_rmse_log[-1]:.4f}")
print(f"Test-RMSE: {test_rmse_log[-1]:.4f}")


#Aufgabe 1.4
momentum_values = [0.5, 0.8, 0.9, 0.99]
results = {}

for momentum in momentum_values:
    print(f"\n Training mit Momentum = {momentum}")
    
    network_momentum = Net()
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(network_momentum.parameters(), lr=learning_rate, momentum=momentum) #SGD mit Momentum 
    
    batch_rmse_log, test_rmse_log = train_model(network_momentum, train_data, test_data, optimizer, criterion, batch_size=batch_size, epochs=n_epochs, log_interval=log_interval)
    
    results[momentum] = {'train': batch_rmse_log[-1], 'test': test_rmse_log[-1]}

print("\nVergleich der finalen RMSEs nach 10 Epochen (mit Momentum):")
for m, values in results.items():
    print(f"Momentum = {m}: Train-RMSE = {values['train']:.4f} | Test-RMSE = {values['test']:.4f}")

#TODO: Vergleichen Sie das Ergebnis und den Verlauf des Lernens mit dem Training ohne Momentum!! (mit 1.3 vergleichen)


#Aufgabe 1.5

optimizers = {'RMSprop': torch.optim.RMSprop, 'Adam': torch.optim.Adam}

results_optimizers = {}

for name, optim_class in optimizers.items():
    print(f"\nTraining mit {name}")
    
    network_adam_and_RMS = Net()
    criterion = torch.nn.MSELoss()
    optimizer = optim_class(network_adam_and_RMS.parameters(), lr=learning_rate)
    
    batch_rmse_log, test_rmse_log = train_model(network_adam_and_RMS, train_data, test_data, optimizer, criterion, batch_size=batch_size, epochs=n_epochs, log_interval=log_interval)
    
    results_optimizers[name] = {'train': batch_rmse_log[-1], 'test': test_rmse_log[-1]}

print("\nVergleich RMSprop & Adam:")
for name, values in results_optimizers.items():
    print(f"{name}: Train-RMSE = {values['train']:.4f} | Test-RMSE = {values['test']:.4f}")

#TODO: Ergebnis mit den beiden vorherigen Methoden vergleichen

/var/folders/jd/nf7qsc150dn4fxlf16c4gs140000gn/T/ipykernel_5016/3479514524.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load("./Daten Blatt 2/electric_mo

Größe des Trainingsdatensatzes: 399228
[Epoche 1 | Batch Nr. 0] RMSE (Train): 1.0047
[Epoche 1 | Batch Nr. 500] RMSE (Train): 0.7414
[Epoche 1 | Batch Nr. 1000] RMSE (Train): 0.7227
[Epoche 1 | Batch Nr. 1500] RMSE (Train): 0.7314
==> Epoche 1: RMSE (Test): 0.7099
[Epoche 2 | Batch Nr. 0] RMSE (Train): 0.6885
[Epoche 2 | Batch Nr. 500] RMSE (Train): 0.7306
[Epoche 2 | Batch Nr. 1000] RMSE (Train): 0.7232
[Epoche 2 | Batch Nr. 1500] RMSE (Train): 0.7060
==> Epoche 2: RMSE (Test): 0.6926
[Epoche 3 | Batch Nr. 0] RMSE (Train): 0.7261
[Epoche 3 | Batch Nr. 500] RMSE (Train): 0.6891
[Epoche 3 | Batch Nr. 1000] RMSE (Train): 0.7217
[Epoche 3 | Batch Nr. 1500] RMSE (Train): 0.6614
==> Epoche 3: RMSE (Test): 0.6799
[Epoche 4 | Batch Nr. 0] RMSE (Train): 0.6656
[Epoche 4 | Batch Nr. 500] RMSE (Train): 0.7034
[Epoche 4 | Batch Nr. 1000] RMSE (Train): 0.6944
[Epoche 4 | Batch Nr. 1500] RMSE (Train): 0.6431
==> Epoche 4: RMSE (Test): 0.6692
[Epoche 5 | Batch Nr. 0] RMSE (Train): 0.7057
[Epoche 5 |

Zu 1.4:


Kategorie|Ohne Momentum|Momentum 0.5 | Momentum 0.8 | Momentum 0.9 | Momentum 0.99
----|-----|-----|-|-|-|
Test RMSE nach 10 Epochen | 0.6075|0.5348|0.4849|0.4620|0.4426
Trainingskurve|flacht ab; kein starkes sinken mehr; Bsp: Veränderung RMSE Epoche 6 auf 7 beträgt: 0,00422 |Kurve fällt stärker; Bsp: Veränderung RMSE Epoche 6 auf 7 beträgt: 0,0312|Kurve fällt stärker; Bsp: Veränderung RMSE Epoche 6 auf 7 beträgt: 0,014|Kurve fällt stärker; Bsp: Veränderung RMSE Epoche 6 auf 7 beträgt: 0,0291| Kurve fällt stärker; Bsp: Veränderung RMSE Epoche 6 auf 7 beträgt: |



Adam erreicht voraussichtlich den niedrigsten Test-RMSE (rund 0.44), vergleichbar mit Momentum = 0.99.

RMSprop liefert auch gute Resultate, ist aber leicht schlechter als Adam und Momentum = 0.99.

Beide modernen Methoden konvergieren schneller als SGD (auch mit Momentum)